James Caldwell <br>
UVA IRA <br>
Jan/Feb 2026
 <br>
This Script is used for automatically filling out the common data set fillable PDF (which meets web accessability requirements) from the excel file.

In [78]:
import pandas as pd
import numpy as np
import pymupdf
from dotenv import load_dotenv
import os
from collections import defaultdict

In [79]:
excel_to_pdf_mapping ={
    #Input:Output #(Excel:PDF)
    'N': 'No',
    'Y': 'Yes',
    'Yes': 'Y',
    'No': 'N',
    'N#2FA': 'N/A',
    "N/A": 'N#2FA',
    'Very Important': 'VI',
    'Important': 'I',
    'Considered': 'C',
    'Not Considered': 'NC',

    "private nonprofit": "Private#20#28nonprofit#29",
    "private (nonprofit)": "Private#20#28nonprofit#29",
    "proprietary": "Proprietary",

    "Coeducational college": "Coeducational#20college",
    "Men's college": "Men's#20college",
    "Womens college": "Women's#20college",

    "Semester": "SEM",
    "Quarter": "QTR",
    "Trimester": "TRI",
    "Continuous": "CON",
    "Differs by program (describe):": "DFR",
    "Other (describe)": "Other",

    "High school diploma is required and GED is accepted": "GED",
    "High school diploma is required and GED is not accepted": "REQ",
    "High school diploma or equivalent is not required": "NON",

    "Recommend":"REC",
    "Require":"REQ",
    "Neither require nor recommend":"Non",

    "Same fee": "SAME",
    "Free": "FREE",
    "Reduced": "RED",

    "Required to be considered for admission":'ADMS_REQ',
    "Recommended":"ADMS_REC",
    "Required for some":"ADMS_RFS#20",
    "Not required for admission, but considered if submitted":"ADMS_CONSIDER#20",
    "Not considered for admission, even if submitted":"ADMS_NOT_USED",

    "Required of All":"TFER_REQ",
    "Recommended of All":"TFER_REC",
    "Recommended of Some":"TFER_RFS",
    "Required of Some":"TFER_ROS",
    "Not Required": "TFER_NREQ",

}



In [ ]:
# Load environment variables
load_dotenv()
# excel_file_path = os.getenv('excel_file_path') 
excel_file_path = os.getenv('testing_file_path') 
checkbox_mapping_file_path = os.getenv('checkbox_mapping_file_path')

# Read the Excel file
df = pd.read_excel(
    excel_file_path,
    sheet_name="Answer Sheet",
    keep_default_na=False
)
doc = pymupdf.open('CDS_2025-2026-PDF_Template_copy.pdf')

checkbox_button_group = defaultdict(list)
non_checkbox_button_group = defaultdict(list)
updated_fields = [] # To track which fields were updated and determined if any not updated correctly
all_fields = [] 
all_fields_options = {}

for page in doc:#doc:
    
    # Load all fillable pdf items, and categorize as checkbox or fill in 
    for widget in page.widgets():

        pdf_field_name = widget.field_name
        all_fields.append(pdf_field_name)
        # what is this?
        if pdf_field_name == "ACCEL_DEG":
                # print('yay found it')
                widget.field_value = 'On'
                widget.update()
                # print('updated')

        
        # print('widget name: ' + widget.field_name)
        # print('widget type: ' + str(widget.field_type))
                # print('widget state: ' + str(widget.field_value))
                # print('widget options: ' + str(widget.button_states()['normal']))
                # print('Excel match: ' + str(excel_match))

        # Here
        all_fields_options[pdf_field_name] = widget.button_states() if widget.field_type == pymupdf.PDF_WIDGET_TYPE_RADIOBUTTON else None

        #0 Collect PDF field tag corresponding value from excel
        try:
            # AD_OPEN and AD_OPEN_T are swapped/mislabeled in the excel file
            if pdf_field_name == "AD_OPEN":
                pdf_field_name = "AD_OPEN_T"
            elif pdf_field_name == "AD_OPEN_T":
                pdf_field_name = "AD_OPEN"
            
            excel_match = df.loc[df["US News PDF Tag"] == pdf_field_name, "Answer"].iloc[0]
            question_number = df.loc[df["US News PDF Tag"] == pdf_field_name, "Question Number"].iloc[0]

            if pdf_field_name == "HOUS_DEPOSIT_REFUND":
                excel_match = "N/A" # N/A is not available in the excel file, but is an option in the PDF, so we will set it to N/A for now

            # All the part E questions outputs X in the excel file, but the PDF expects Y
            if 'E' in question_number or 'F' in question_number:
                if str(excel_match).upper() == 'X':
                    excel_match = 'Y'

            # if pdf_field_name == "ACCEL_DEG":
            #     # excel_match = "N/A"
            #     excel_match = 'Y'
            #     print('widget name: ' + widget.field_name)
            #     print('widget options: ' + str(widget.button_states()))#['normal']))
            #     print(excel_match)
            #     print(widget.field_type)

            #1 Skip empty fields
            if pd.isna(excel_match) and excel_match != "N/A":
                updated_fields.append(pdf_field_name) # Count as updated even if skipped. Trying to track which ones we have data for but did not update correctly
                continue
        
            #2 Checkbox button
            elif widget.field_type == pymupdf.PDF_WIDGET_TYPE_RADIOBUTTON:

                # if pdf_field_name == "AD_OPEN_T":
                #     print('boo')
                    # print('widget name: ' + widget.field_name)
                    # print('widget options: ' + str(widget.button_states()['normal']))
                    # print(excel_match)
                # if pdf_field_name in missing_fields:

                #     # print('widget state: ' + str(widget.field_value))
                # print('widget options: ' + str(button_option))
                # print('widget name: ' + widget.field_name)
                # print('Excel match: ' + str(excel_match))


                excel_match = str(excel_match).strip()
                button_option = widget.button_states()['normal'] # list such as ['Public Institution', 'Off']
                if excel_match in button_option:
                    try:
                        widget.field_value = 'On'
                        widget.update()
                        updated_fields.append(pdf_field_name)
                    except Exception as e:
                        print(f"Error updating radio button field {pdf_field_name} with value {excel_match}: {e}")
                else:

                    
                    # Try mapping Excel value to PDF value, then try updating again
                        # PDF labeling is inconsistent, sometimes yes is "Yes" and sometimes "Y"
                        # Above try block attempts direct match first, this block attempts mapped match
                    excel_match = excel_to_pdf_mapping.get(excel_match, excel_match)

                    if excel_match in button_option:

                        try:
                            widget.field_value = 'On'
                            widget.update()
                            updated_fields.append(pdf_field_name)
                        except Exception as e:
                            print(f"Error updating radio button field {pdf_field_name} with value {excel_match}: {e}")
                    # else:
                    #     print('no match for radio button:')
                
            #3 Fillable text/number fields
            else:
                excel_match = str(excel_match).strip()
                # print(f"{pdf_field_name}: {excel_match}")
                try:
                    widget.field_value = excel_match
                    widget.update()
                    updated_fields.append(pdf_field_name)
                except Exception as e:
                    print(f"Error updating field {pdf_field_name} with value {excel_match}: {e}")
        except IndexError:
            print(f"No match found for field: {pdf_field_name}")

missing_fields = [field for field in all_fields if field not in updated_fields]
print("Fields not updated (missing data or update error):")
for field in missing_fields:
    print(field)

doc.save(r"CDS_2025-2026_filled.pdf")
doc.close()

No match found for field: URL_DESTINATION
No match found for field: URL_DESTINATION
Fields not updated (missing data or update error):
URL_DESTINATION
URL_DESTINATION
AD_OPEN_MOST
AD_OPEN_MOST
AD_OPEN_T_CHECK
AD_OPEN_MOST
AD_OPEN_T_CHECK
EXAM_CODE_ACT
EXAM_CODE_ACT
EXAM_CODE_ACT
EXAM_CODE_ACT
EXAM_CODE_ACT
EXAM_CODE_SAT
EXAM_CODE_SAT
EXAM_CODE_SAT
EXAM_CODE_SAT
EXAM_CODE_SAT
ROTC_ARMY
ROTC_ARMY
ROTC_MARINE_OPT
ROTC_NAVY
ROTC_NAVY
ROTC_AF
ROTC_AF
ACAD_YR
ACAD_YR
ACAD_YR
